<a href="https://colab.research.google.com/github/despiegj/goz39a/blob/main/Data_Preprocessing_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing with Pandas
Preprocessing is the process of doing a pre-analysis of data, in order to transform them into a standard and normalized format.
Preprocessing involves the following aspects:
- missing values
- data formatting
- data standardization
- data normalization
- data binning

References : 
- https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-1-missing-data-45e76b781993
- https://towardsdatascience.com/data-processing-with-python-pandas-part-2-data-formatting-710c2eafa426


## Missing Values
The `hepatitis` dataset was downloaded from [openML](https://www.openml.org/d/55)



In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://goz39a.s3.eu-central-1.amazonaws.com/hepatitis_csv.csv')
df.head(10)

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,30,male,False,False,False,False,False,False,False,False,False,False,False,1.0,85.0,18.0,4.0,NaN,False,live
1,50,female,False,False,True,False,False,False,False,False,False,False,False,0.9,135.0,42.0,3.5,NaN,False,live
2,78,female,True,False,True,False,False,True,False,False,False,False,False,0.7,96.0,32.0,4.0,NaN,False,live
3,31,female,NaN,True,False,False,False,True,False,False,False,False,False,0.7,46.0,52.0,4.0,80.0,False,live
4,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live
5,34,female,True,False,False,False,False,True,False,False,False,False,False,0.9,95.0,28.0,4.0,75.0,False,live
6,51,female,False,False,True,False,True,True,False,True,True,False,False,NaN,NaN,NaN,NaN,NaN,False,die
7,23,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,NaN,NaN,NaN,False,live
8,39,female,True,False,True,False,False,True,True,False,False,False,False,0.7,NaN,48.0,4.4,NaN,False,live
9,30,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,120.0,3.9,NaN,False,live


We note that the dataset presents some problems. For example, the column email is not available for all the rows. In some cases it presents the `NaN `value, which means that the value is missing.
In order to check whether our dataset contains missing values, we can use the function `isna()`, which returns if an cell of the dataset if NaN or not. Then we can count how many missing values there are for each column.

In [4]:
df.isna().sum()

age                 0
sex                 0
steroid             1
antivirals          0
fatigue             1
malaise             1
anorexia            1
liver_big          10
liver_firm         11
spleen_palpable     5
spiders             5
ascites             5
varices             5
bilirubin           6
alk_phosphate      29
sgot                4
albumin            16
protime            67
histology           0
class               0
dtype: int64

When dealing with missing values, different alternatives can be applied:

1. drop missing values
2. replace the missing value with a value
3. leave the missing value as it is.




### 1. Drop Missing Values

Dropping missing values can be one of the following alternatives:
- remove rows having missing values
- remove the whole column containing missing values

We can use the dropna() by specifying the axis to be considered. If we set **axis = 0** we drop the entire **row**, if we set **axis = 1** we drop the whole **column**. 

If we apply the function `df.dropna(axis=0)` 80 rows of the dataset remain. If we apply the function `df.dropna(axis=1)`, only the columns age, sex, antivirals, histology and class remain. 

However, removed values are not applied to the original dataframe, but only to the result! We can use the argument inplace=True in order to store changes in the original dataframe df (df.dropna(axis=1,inplace=True)).

In [5]:
print('Dimensions of the dataset:',df.shape)
print('Dimensions after removing columns:',df.dropna(axis=1).shape)

Dimensions of the dataset: (155, 20)
Dimensions after removing columns: (155, 5)


Another alternative involves the dropping of columns where a certain percentage of not-null values is available. This can be achieved through the `thresh` parameter. In the following example we keep only columns where there are at least the 80% of not null values.

In [6]:
dg = df.dropna(thresh=0.8*len(df),axis=1)
dg.shape

(155, 19)

### 2. Replace missing values
A good strategy when dealing with missing values involves their replacement with another value. Usually, the following strategies are adopted:
- for numerical values replace the missing value with the average value of the column
- for categorial values replace the missing value with the most frequent value of the column
- use other functions

In order to replace missing values, several functions can be used: `fillna()`, `replace()` and `interpolate()`. 

The fillna() function replaces all the NaN values with the value passed as argument. For example, for numerical values, all the NaN values in the numeric columns could be replaced with the average value. Another possibility is to used the `ffill` and `bfill` method in combination with the fillna()-function.

In order to list the type of a column, we can use the attribute dtypes as follows:

In [7]:
df.dtypes

age                  int64
sex                 object
steroid             object
antivirals            bool
fatigue             object
malaise             object
anorexia            object
liver_big           object
liver_firm          object
spleen_palpable     object
spiders             object
ascites             object
varices             object
bilirubin          float64
alk_phosphate      float64
sgot               float64
albumin            float64
protime            float64
histology             bool
class               object
dtype: object

#### Numerical Columns

In [8]:
import numpy as np
numeric = df.select_dtypes(include=np.number)
numeric_columns = numeric.columns
print(numeric_columns)

Index(['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin', 'protime'], dtype='object')


In [9]:
df[numeric_columns] = df[numeric_columns].fillna(df.mean())

Check the result

In [10]:
df.isna().sum()

age                 0
sex                 0
steroid             1
antivirals          0
fatigue             1
malaise             1
anorexia            1
liver_big          10
liver_firm         11
spleen_palpable     5
spiders             5
ascites             5
varices             5
bilirubin           0
alk_phosphate       0
sgot                0
albumin             0
protime             0
histology           0
class               0
dtype: int64

#### Categorical Columns

We note that in dtypes the categorial columns are described as objects. Thus we can select the object columns. We would like to consider only boolean columns. However the object type includes also the column class, which is a string. We select all the object columns, and then we remove from them the column class. Then we can convert the type of the result to bool.MM

In [11]:
boolean_columns = df.select_dtypes(include=np.object).columns.tolist()
boolean_columns.remove('class')
print(boolean_columns)
df[boolean_columns] = df[boolean_columns].astype('bool')

['sex', 'steroid', 'fatigue', 'malaise', 'anorexia', 'liver_big', 'liver_firm', 'spleen_palpable', 'spiders', 'ascites', 'varices']


Now we can replace all the missing values for booleans with the most frequent value. We can use the mode() function to calculate the most frequent value. We use the fillna() function to replace missing values, but we could use also the replace(old_value,new_value) function.



In [12]:
df[boolean_columns]= df[boolean_columns].fillna(df.mode())

In [13]:
df.isna().sum()

age                0
sex                0
steroid            0
antivirals         0
fatigue            0
malaise            0
anorexia           0
liver_big          0
liver_firm         0
spleen_palpable    0
spiders            0
ascites            0
varices            0
bilirubin          0
alk_phosphate      0
sgot               0
albumin            0
protime            0
histology          0
class              0
dtype: int64

## Data Formatting

Data formatting is the process of transforming data into a common format, which helps users to perform comparisons. An example of not formatted data is the following: the same entity is referred in the same column with different values, such as **New York** and NY.

Firstly, import data using the pandas library and convert them into a dataframe. Through the head(10) method we print only the first 10 rows of the dataset.


In [17]:
df = pd.read_csv('https://goz39a.s3.eu-central-1.amazonaws.com/sample_tweets/sample.csv')
print('Dimension of the dataframe:',df.shape)
df.head()

Dimension of the dataframe: (386, 21)


,Tweet Id,Tweet URL,Tweet Posted Time (UTC),Tweet Content,Tweet Type,Client,Retweets Received,Likes Received,Tweet Location,Tweet Language,User Id,Name,Username,User Bio,Verified or Non-Verified,Profile URL,Protected or Non-protected,User Followers,User Following,User Account Creation Date,Impressions
0,"""1167429261210218497""",https://twitter.com/animalhealthEU/status/1167...,30 Aug 2019 13:30:00,Pets change our lives &amp; become a part of o...,Tweet,Twitter Ads Composer,0,4,Brussels,English,"""1017044760""",AnimalhealthEurope,animalhealthEU,AnimalhealthEurope represents manufacturers of...,Non-Verified,https://twitter.com/animalhealthEU,Non-Protected,3697,542,17 Dec 2012 09:14:15,7394
1,"""1167375334670557185""",https://twitter.com/PennyBrohnUK/status/116737...,30 Aug 2019 09:55:43,Another spot of our #morethanmedicine bus in #...,Tweet,Twitter Web App,0,5,"Pill, Bristol",English,"""190981222""",Penny Brohn UK,PennyBrohnUK,We help people live well with the impact of ca...,Non-Verified,https://twitter.com/PennyBrohnUK,Non-Protected,3227,1571,15 Sep 2010 09:44:02,6454
2,"""1167237977615097861""",https://twitter.com/lordbyronaf/status/1167237...,30 Aug 2019 00:49:54,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,Twitter for Android,0,0,"Ohio, USA",English,"""716285395""",Lord ByronAF,lordbyronaf,"It's easier to be who you are, than it is to b...",Non-Verified,https://twitter.com/lordbyronaf,Non-Protected,7808,8617,25 Jul 2012 15:43:47,0
3,"""1167236897078480898""",https://twitter.com/CountessDavis/status/11672...,30 Aug 2019 00:45:37,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,Twitter for Android,0,0,NaN,English,"""824683748964851712""",Lisa Countess davis,CountessDavis,I am named after @ElvisPresley daughter Lisa M...,Non-Verified,https://twitter.com/CountessDavis,Non-Protected,291,81,26 Jan 2017 18:21:42,0
4,"""1167228378191204353""",https://twitter.com/Local12/status/11672283781...,30 Aug 2019 00:11:46,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,TweetDeck,0,0,"Cincinnati, OH",English,"""16102208""",Local 12/WKRC-TV,Local12,Local 12 is #Cincinnati's trusted source for b...,Verified,https://twitter.com/Local12,Non-Protected,198675,651,02 Sep 2008 20:09:44,0


Drop all the missing values through the dropna() function.

In [18]:
df.dropna(how='any',inplace=True)

Examin the different Data Types

In [33]:
df.dtypes

Tweet Id                      object
Tweet URL                     object
Tweet Posted Time (UTC)       object
Tweet Content                 object
Tweet Type                    object
Client                        object
Retweets Received              int64
Likes Received                 int64
Tweet Location                object
Tweet Language                object
User Id                       object
Name                          object
Username                      object
User Bio                      object
Verified or Non-Verified      object
Profile URL                   object
Protected or Non-protected    object
User Followers                 int64
User Following                 int64
User Account Creation Date    object
Impressions                    int64
dtype: object

Quick update on the characteristics of the numerical data

In [34]:
df.describe()

,Retweets Received,Likes Received,User Followers,User Following,Impressions
count,317.000000,317.000000,317.000000,317.000000,317.000000
mean,0.794953,3.195584,2748.217666,1086.637224,1222.769716
std,1.943564,10.137640,11563.933842,1437.933564,2496.502317
min,0.000000,0.000000,2.000000,4.000000,0.000000
25%,0.000000,0.000000,441.000000,379.000000,0.000000
50%,0.000000,0.000000,866.000000,542.000000,0.000000
75%,0.000000,1.000000,3167.000000,1123.000000,882.000000
max,13.000000,98.000000,198675.000000,12476.000000,12033.000000


In our case we can convert the column Tweet Location to string by using the function astype() as follows:

In [35]:
df['Tweet Location'] = df['Tweet Location'].astype('string')

In [36]:
df.dtypes

Tweet Id                      object
Tweet URL                     object
Tweet Posted Time (UTC)       object
Tweet Content                 object
Tweet Type                    object
Client                        object
Retweets Received              int64
Likes Received                 int64
Tweet Location                string
Tweet Language                object
User Id                       object
Name                          object
Username                      object
User Bio                      object
Verified or Non-Verified      object
Profile URL                   object
Protected or Non-protected    object
User Followers                 int64
User Following                 int64
User Account Creation Date    object
Impressions                    int64
dtype: object

### Make the data homogeneous
This aspect involves categorical and numerical data. 
- **Categorical** data should have all the same formatting style, such as lower case. For text mining applications there are other formatting interventions:
   - stop word removal
   - lemmatization
   - punktuation removal
   - ....
- **Numerical** data should have for example the same number of digits after the point. 

In order to format all categorical data to lower case, we can use the following statement: